In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

In [2]:
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [105]:
import os
import sys

if os.path.abspath('../..') not in sys.path:
    sys.path.append(os.path.abspath('../..'))

In [133]:
import hashlib

import numpy as np
import pandas as pd

from src.dataset import load_dataset

In [137]:
df = load_dataset()

In [142]:
df[['subject_id', 'participant']] \
    .drop_duplicates() \
    .reset_index(drop=False)['participant'] \
    .value_counts()

True     1489
False    1483
Name: participant, dtype: int64